In [ ]:
# Install the concept discovery library
#!git clone https://github.com/maragraziani/concept_discovery_svd.git

# Download the demo dataset at https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
#!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz

#import sys
#sys.path.append('concept_discovery_svd/')

In [2]:
import os
import sys
sys.path.append(os.getcwd().split('/notebooks')[0])

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import scripts.datasets as datasets
import torch
import torchvision.models
import numpy as np
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

In [4]:
import scripts.cdisco.cdisco as cdisco
import scripts.cdisco.analyze as analyze
import scripts.cdisco.utils as utils
import scripts.cdisco.vis as vis

In [5]:
# Load dataset

dataset='imagenette'
data_folder = '/home/mara/discovery'
source = f'{data_folder}/{dataset}2-320/train'
train_files_path = (source+'/train')
paths, count, y, idx_to_labels = datasets.get_dataset(dataset, source=source)

print(count, len(paths))

5000 5000


In [6]:
# Load model
model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)
model.eval()

Using cache found in /home/mara/.cache/torch/hub/pytorch_vision_v0.9.0
/home/mara/torchenv2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mara/torchenv2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [10]:
# Select a layer and create a saving directory 

try:
    os.mkdir('outputs')
except:
    print("Saving in outputs")
layer='Mixed_7b.cat_2'
SAVEFOLD=f'outputs/{dataset}'
if not os.path.exists(SAVEFOLD):
    os.mkdir(SAVEFOLD)
try:
    os.mkdir(f"{SAVEFOLD}/{layer}/")
except:
    print(f"Maybe the directory already exists? Overriding results in {SAVEFOLD}/{layer}")
SAVEFOLD=f"{SAVEFOLD}/{layer}/"

Saving in outputs
Maybe the directory already exists? Overriding results in outputs/imagenette/Mixed_7b.cat_2


In [11]:
classes = np.unique(y)

In [12]:
print(layer)
return_nodes={f'{layer}': 'conv', 'avgpool':'avgpool', 'fc':'fc'
             }
model = torchvision.models.feature_extraction.create_feature_extractor(model, return_nodes=return_nodes)

Mixed_7b.cat_2


/home/mara/torchenv2/lib/python3.10/site-packages/torchvision/models/feature_extraction.py:174: UserWarning: NOTE: The nodes obtained by tracing the model in eval mode are a subsequence of those obtained in train mode. When choosing nodes for feature extraction, you may need to specify output nodes for train and eval mode separately.
  warnings.warn(msg + suggestion_msg)


In [13]:
# Getting the dimensions of the feature space
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inputs=torch.zeros((8,3,299,299), dtype=torch.float).to(device)
model.to(device)
outs=model(inputs)
dim_c = outs['conv'].shape[1]
dim_w = outs['conv'].shape[2]
dim_h = outs['conv'].shape[3]

In [14]:
print("Feature space dimensions: ",dim_c, dim_w, dim_h)

Feature space dimensions:  2048 8 8


In [16]:
""" 
Extract model state: Use to extrac the model gradients, predictions and activations for the loaded dataset

"""
#cdisco.get_model_state(model, paths, y, dim_c, dim_w, dim_h, SAVEFOLD=SAVEFOLD)

"""
LOAD model state results: Use to load a pre-computed model state
"""
LOADFOLD = '/home/mara/discovery/demo/outputs/imagenette/Mixed_7b.cat_2/'
gradients_wrt_conv_layer=np.load(f"{LOADFOLD}/gradients_wrt_conv_layer.npy")
predictions=np.load(f"{LOADFOLD}/predictions.npy")
conv_maps=np.load(f"{LOADFOLD}/conv_maps.npy")

"""
RUN Concept Discovery with SVD
"""
class_concept_candidates, pvh = cdisco.cdisco(conv_maps, gradients_wrt_conv_layer, predictions, classes)

np.save(f"{SAVEFOLD}/cdisco_candidates.npy", class_concept_candidates)
np.save(f"{SAVEFOLD}/eigenvectors.npy", pvh)

In [18]:
concepts, candidates=cdisco.cdisco_concepts_list(class_concept_candidates,classes,limit=1)

In [19]:
print("Concept vector list: ", concepts)
print("Classes correspnding to each concept vector: ", candidates)

Concept vector list:  {357, 453, 1574, 504, 211, 1172, 179, 440, 763, 542}
Classes correspnding to each concept vector:  {357: [0.0], 453: [569.0], 1574: [701.0], 504: [482.0], 211: [497.0], 1172: [571.0], 179: [574.0], 440: [217.0], 763: [491.0], 542: [566.0]}


In [20]:
print("Detailed list of concept vectors found for each class")
class_concept_candidates

Detailed list of concept vectors found for each class


{0: array([ 357,  400,  338, ..., 1435,  169,  269]),
 217: array([ 440,  347,  995, ..., 2044, 1336, 1517]),
 482: array([ 504, 1322,  735, ..., 1887,  412, 1462]),
 491: array([ 763, 1172, 1843, ..., 1527,  253, 1322]),
 497: array([ 211, 2020,  424, ..., 1172, 1147,  124]),
 566: array([ 542, 1194,  412, ..., 1120, 2034,  750]),
 569: array([ 453,  246, 1480, ...,  487,  808,  378]),
 571: array([1172, 2038,  760, ..., 1892, 1297,  789]),
 574: array([ 179, 1172,  938, ..., 1846,  789, 1120]),
 701: array([1574, 1172, 1928, ...,  789, 1838, 1846])}

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import scipy

plt.rcParams['figure.figsize']=(40,20)

all_images=[]
for c in concepts:
    c=int(c)
    concept_vector = pvh[c,:]  
    classes_with_concept = candidates[c][:]
    image_idxs=[]
    class_iterator=0
    image_iterator=0
    while len(image_idxs)<10:
        cwc=classes_with_concept[class_iterator]
        cc_= np.argwhere(int(cwc)==np.argmax(predictions,axis=1))
        
        image_idxs.append(cc_[image_iterator].ravel()[0])
        class_iterator+=1
        if (class_iterator==len(classes_with_concept)):
            image_iterator+=1
            class_iterator=0
    
    i=1
    
    images = image_idxs[:10]
    all_images.append(images)
    plt.figure()
    for img_id in images[:20]:
        plt.subplot(2,len(images),i)
        fmap=vis.cdisco_concept_vis(img_id, concept_vector, dim_c, dim_w, dim_h, conv_maps)
        
        plt.imshow((datasets.transform(Image.open(paths[img_id]))).swapaxes(0,1).swapaxes(1,-1))#,alpha=0.5)
        hmap=scipy.ndimage.zoom(fmap, 299/fmap.shape[0],order=1)
        th = np.percentile(hmap,85) 
        plt.imshow((hmap>th)*hmap, cmap='PiYG_r', alpha=0.4, vmin=-np.abs(hmap).max(), vmax=np.abs(hmap).max())
        plt.axis("off")
        i+=1
    
    plt.savefig(f"{SAVEFOLD}_concept_map_{c}_everything_v1.png")

In [ ]:
# Isolating polysemantic concepts and refining the vectors to obtain unique concept vectors

activations=[]
images = all_images[2]
for img_id in images:
    activations.append(conv_maps[img_id])
activations = np.asarray(activations)
#activations = np.concatenate(activations)

In [ ]:
all_images, images, img_id, 

In [ ]:
from sklearn.cluster import KMeans
import math


kmeans = KMeans(n_clusters=2, random_state=0, n_init=5, max_iter=1000).fit(np.mean(activations, axis=(2,3)))
clu_labs = kmeans.labels_
clu_lab_order = sorted(range(len(clu_labs)), key=lambda k: clu_labs[k])

idx = 0
for idx, im_id in enumerate(images):
    plt.figure()
    plt.imshow((datasets.transform(Image.open(paths[im_id]))).swapaxes(0,1).swapaxes(1,-1))#,alpha=0.5
    plt.title(f"{im_id}: cluster {clu_labs[clu_lab_order][idx]}", size = 8)
    plt.axis('off')


In [ ]:
clu_labs

In [ ]:
import umap.umap_ as umap
from matplotlib.offsetbox import OffsetImage, AnnotationBbox


XY_UMAP = umap.UMAP(n_components=2).fit_transform(np.mean(activations, axis=(2,3)))

def getImage(path, zoom=0.8):
    return OffsetImage(plt.imread(path), zoom=zoom)

def rand_jitter(arr, amount):
    stdev = amount * (max(arr) - min(arr))
    return arr + np.random.randn(len(arr)) * stdev

def jitter(x, y, ax, s=20, c='b', marker='o', cmap=None, norm=None,
           vmin=None, vmax=None, alpha=None, linewidths=None,
           verts=None, hold=None, **kwargs):
    return ax.scatter(rand_jitter(x, amount), rand_jitter(y, amount),
                      s=s, c=c, marker=marker, cmap=cmap,
                      norm=norm, vmin=vmin, vmax=vmax, alpha=alpha,
                      linewidths=linewidths, **kwargs)

In [ ]:
clu1 = []
clu2 = []
i=0
for im in images:
    if clu_labs[i]==1:
        clu1.append(np.mean(conv_maps[im,:,:], axis=(1,2)))
    else:
        clu2.append(np.mean(conv_maps[im,:,:], axis=(1,2))   )         
    i+=1


In [ ]:
v1 = np.mean(np.asarray(clu1), axis=0)
v2 = np.mean(np.asarray(clu2),axis=0)

In [ ]:
v1.shape, v2.shape, len(clu1), clu1[0].shape


In [ ]:

plt.rcParams['figure.figsize']=(40,20)

clu_labs = kmeans.labels_
clu_lab_order = sorted(range(len(clu_labs)), key=lambda k: clu_labs[k])


fig, ax = plt.subplots(figsize = (10,10))
amount=0.
ax.scatter(XY_UMAP[:,0], XY_UMAP[:,1]) 
top_ims=images
bool_cluster=False 
plt.figure()

i=0
for x0, y0, path in zip(rand_jitter(XY_UMAP[:,0], amount), rand_jitter(XY_UMAP[:,1], amount), [paths[i] for i in top_ims]):
    bool_cluster=False
    if clu_labs[i]==0: 
        bool_cluster=True 
    ab = AnnotationBbox(getImage(path, zoom=0.2), (x0, y0), frameon=bool_cluster)
    ax.add_artist(ab)
    i+=1
